#### This Notebook will prompt you for your ArcGIS Online credentials and use them to query items in your org.
#### The credentials you provide must have Admin privileges on your org.

In [1]:
from arcgis.gis import *
from arcgis.mapping import WebMap
import getpass, datetime
import pandas as pd
import syswvr4

**item_df_columns** is a list of column names that will appear in the exported spreadsheet.

**excel_file** is the name and location of the exported excel file.

**url_key_word** is key word that you will search for within item URLs in your org.

**agol_user** is the user who is logging into AGOL.  This user must have admin privileges within the org.
You will be prompted to enter this value.

**agol_pass** is the password of the admin user who is logging in.

In [2]:
item_df_columns = ['Login_Name', 'Full_Name', 'Email_Address', 'Folder', 'Item_Title', 'Item_Type', 'Service URL']
excel_file = 'C:/temp/FirstMap_URL_Report_Pro.xlsx'
url_key_word = 'firstmap'  # this notebook is currently set to look for FirstMap urls.  Change this to look for something else.

In [23]:
agol_user = input('Enter User Name: ')

Enter User Name: mtownshend@de


In [24]:
agol_pass = getpass.getpass('Enter Password: ')

Enter Password: ········


In [25]:
# Creating the excel writer object.
writer = pd.ExcelWriter(excel_file, engine = 'openpyxl')

In [26]:
# Tis function gets all users in the org.
def get_users(user, user_pass):

    gis = GIS(url='https://www.arcgis.com', username = user, password = user_pass)
    
    users = gis.users.search('!esri_ & !admin', max_users = 250)
    
    return users

In [27]:
# This funciton will look at urls in each web map and write a line for 
# each instance where a URL contains the url_key_word set above.
def make_map_report(users, report_name):
    list=[]
    errors=[]
    
    for agol_user in users:
        user_content = agol_user.items()
        # gets info for items not in a folder.
        for item in user_content:
            if item.type == 'Web Map':
                web_map = WebMap(item)
                for l in web_map.layers:
                    try:
                        if url_key_word in l.url:
                            list.append([agol_user.username, 
                                    agol_user.fullName, 
                                    agol_user.email,
                                    None,
                                    item.title,
                                    item.type,
                                    l.url])
                    except Exception:
                        e = sys.exc_info()[1]
                        errors.append([e])

        folders = agol_user.folders

        # Get info for items in a folder.
        for folder in folders:
            folder_items = agol_user.items(folder=folder['title'])
            for item in folder_items:
                if item.type == 'Web Map':
                    web_map = WebMap(item)
                    for l in web_map.layers:
                        try:
                            if url_key_word in l.url:
                                list.append([agol_user.username, 
                                        agol_user.fullName, 
                                        agol_user.email,
                                        folder['title'],
                                        item.title,
                                        item.type,
                                        l.url])
                        except Exception:
                            e = sys.exc_info()[1]
                            errors.append([e])

    item_df = pd.DataFrame(list, columns=item_df_columns)
    item_df.to_excel(writer, sheet_name = report_name)

In [28]:
# This funciton will look at urls in each feature service and write a line for 
# each instance where a URL contains the url_key_word set above.
def make_feature_service_report(users, report_name):
    layer_list = []
    for agol_user in users:
        user_content = agol_user.items()
        # gets info for items not in a folder.
        for item in user_content:
            if item.type == 'Feature Service':
                    if url_key_word in item.url:
                        layer_list.append([agol_user.username, 
                                    agol_user.fullName, 
                                    agol_user.email,
                                    None,
                                    item.title,
                                    item.type,
                                    item.url])


        folders = agol_user.folders

        # Get info for items in a folder.
        for folder in folders:
            folder_items = agol_user.items(folder=folder['title'])
            for f_item in folder_items:
                if f_item.type == 'Feature Service':
                    if url_key_word in f_item.url:
                            layer_list.append([agol_user.username, 
                                    agol_user.fullName, 
                                    agol_user.email,
                                    folder['title'],
                                    f_item.title,
                                    f_item.type,
                                    f_item.url])

    # dump the layer_list into a data frame
    layer_df = pd.DataFrame(layer_list, columns=item_df_columns)
    # Export the data frame to an Excel Spreadsheet
    layer_df.to_excel(writer, sheet_name = report_name)

In [29]:

# Get all AGOL Users in your Org
users = get_users(agol_user, agol_pass)

# Make the report for all Web Maps
make_map_report(users, 'Web Map Report')

# Make the report for all Feature Services
make_feature_service_report(users, 'Feature Service Report')


In [30]:
# save and close the Excel File.  You may not be able to open it unless you save and close.
writer.save()
writer.close() 